# agrobr — Demo Completo

Dados agrícolas brasileiros em uma linha de código.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bruno-portfolio/agrobr/blob/main/examples/agrobr_demo.ipynb)

Este notebook demonstra as principais funcionalidades do **agrobr v0.9.0**:
- 13 fontes de dados com golden tests (CEPEA, CONAB, IBGE, NASA POWER, BCB/SICOR, ComexStat, ANDA, ABIOVE, USDA, IMEA, DERAL, INMET, Notícias Agrícolas)
- Camada semântica com fallback automático
- MetaInfo para proveniência e rastreabilidade (attempted_sources, selected_source)
- Resiliência HTTP: retry centralizado, 429 handling, Retry-After
- Cache DuckDB inteligente com histórico permanente
- Modo síncrono e async

> **Nota:** INMET requer token de autenticação (`AGROBR_INMET_TOKEN`). Sem token, dados retornam vazio (HTTP 204). Use **NASA POWER** como alternativa sem autenticação — seção 10 deste notebook.

## 1. Instalação

In [ ]:
!pip install -q agrobr

In [ ]:
## 1b. Configurar INMET Token (opcional)

# A API de dados observacionais do INMET requer token de autenticacao.
# Sem token, requisicoes retornam HTTP 204 (sem conteudo).
# Para dados climaticos sem token, use NASA POWER (secao 10).

import os

os.environ["AGROBR_INMET_TOKEN"] = "seu-token-aqui"  # Substitua pelo seu token real

# Para obter o token: https://portal.inmet.gov.br
# Sem token, listagem de estacoes funciona, mas dados observacionais nao.

## 2. Preço Diário — Camada Semântica

O `datasets.preco_diario()` resolve a melhor fonte automaticamente (CEPEA → Notícias Agrícolas → cache).

In [ ]:
from agrobr.sync import datasets

df_preco = datasets.preco_diario("soja", inicio="2025-01-01")
print(f"Registros: {len(df_preco)}, Colunas: {df_preco.columns.tolist()}")
df_preco.head(10)

## 3. MetaInfo — Proveniência

Toda consulta pode retornar `MetaInfo` com rastreabilidade completa da origem dos dados.

In [ ]:
df_preco, meta = datasets.preco_diario("soja", inicio="2025-01-01", return_meta=True)

print("=== MetaInfo — Proveniencia Auditavel ===")
print(f"  source:            {meta.source}")
print(f"  selected_source:   {meta.selected_source}")
print(f"  attempted_sources: {meta.attempted_sources}")
print(f"  dataset:           {meta.dataset}")
print(f"  contract_version:  {meta.contract_version}")
print(f"  schema_version:    {getattr(meta, 'schema_version', 'N/A')}")
print(f"  fetch_timestamp:   {getattr(meta, 'fetch_timestamp', 'N/A')}")
print(f"  records_count:     {meta.records_count}")
print(f"  from_cache:        {meta.from_cache}")
print(f"  fetch_duration_ms: {meta.fetch_duration_ms}")
print(f"  agrobr_version:    {meta.agrobr_version}")
print()
print("Quando fallback ocorre, attempted_sources mostra a cadeia percorrida.")
print("selected_source indica qual fonte efetivamente forneceu os dados.")

## 4. Fallback Automático

O dataset resolve sozinho quando a fonte primária falha.
Observe `attempted_sources` — se CEPEA retornar 403, cai para Notícias Agrícolas ou cache.

In [ ]:
# O fallback e transparente: voce pede o dado, o agrobr resolve a fonte
df, meta = datasets.preco_diario("milho", inicio="2025-01-01", return_meta=True)

print("Produto: milho")
print(f"Fontes tentadas: {meta.attempted_sources}")
print(f"Fonte selecionada: {meta.selected_source}")
print(f"Registros: {meta.records_count}")
df.head(5)

## 5. Indicadores CEPEA

Acesso direto aos indicadores CEPEA/ESALQ — 20 produtos disponíveis.

In [ ]:
from agrobr.sync import cepea

# Listar todos os produtos disponiveis
produtos = cepea.produtos()
print(f"Produtos CEPEA ({len(produtos)}): {produtos}")

In [ ]:
# Serie historica de cafe
df_cafe = cepea.indicador("cafe", inicio="2025-01-01")
print(f"Cafe: {len(df_cafe)} registros")
df_cafe.head(5)

## 6. CONAB — Safras

Estimativas de safra da CONAB. Requer acesso à página de boletins (pode falhar em ambientes sem Playwright).

In [ ]:
from agrobr.sync import conab

try:
    df_safra = conab.safras("soja", safra="2024/25")
    print(f"Safra 2024/25: {len(df_safra)} UFs")
    display(df_safra[["uf", "area_plantada", "producao", "produtividade"]].head(10))
except Exception as e:
    print(f"CONAB safras indisponivel: {type(e).__name__}")
    print("Dica: em ambientes restritos, instale playwright (pip install agrobr[browser])")

## 7. IBGE — PAM

Produção Agrícola Municipal (anual) do IBGE/SIDRA.

In [ ]:
from agrobr.sync import ibge

df_pam = ibge.pam("soja", ano=2023, nivel="uf")
print(f"PAM soja 2023: {len(df_pam)} registros, colunas: {df_pam.columns.tolist()}")
df_pam.head(10)

## 8. Comparar Fontes — CONAB vs IBGE

O agrobr permite cruzar dados de diferentes fontes para validação.

In [ ]:
# IBGE PAM 2023 - producao por UF
# Coluna pode ser "producao" (apos pivot) ou "Quantidade produzida" (nome SIDRA original)
col_prod = "producao" if "producao" in df_pam.columns else "Quantidade produzida"
col_local = "localidade" if "localidade" in df_pam.columns else df_pam.columns[0]

if col_prod in df_pam.columns:
    top_ibge = df_pam[[col_local, col_prod]].dropna(subset=[col_prod]).nlargest(5, col_prod)
    print("=== Top 5 UFs - IBGE PAM 2023 (Toneladas) ===")
    print(top_ibge.to_string(index=False))
else:
    print(f"Coluna de producao nao encontrada. Colunas disponiveis: {df_pam.columns.tolist()}")
    print(df_pam.head())

# CONAB safra 2023/24
try:
    df_conab = conab.safras("soja", safra="2023/24")
    top_conab = df_conab[["uf", "producao"]].sort_values("producao", ascending=False).head(5)
    print("\n=== Top 5 UFs - CONAB (safra 2023/24, mil ton) ===")
    print(top_conab.to_string(index=False))
except Exception as e:
    print(f"\nCONAB indisponivel: {type(e).__name__}")

## 9. Balanço Oferta & Demanda

Balanço de suprimentos da CONAB (oferta, demanda, estoques).

In [ ]:
try:
    df_balanco = conab.balanco("soja")
    if len(df_balanco) > 0:
        print(f"Balanco soja: {len(df_balanco)} registros")
        display(df_balanco.head(10))
    else:
        print("Balanco retornou vazio (formato do boletim pode ter mudado)")
except Exception as e:
    print(f"Balanco indisponivel: {type(e).__name__}")
    print("Dica: balanco depende do XLSX da CONAB, que pode mudar de formato entre levantamentos")

## 10. NASA POWER — Climatologia

Dados climáticos via satélite NASA POWER. Resolução ~55 km (grid 0.5°), cobertura global, desde 1981.
API REST pura, sem autenticação, 7 parâmetros agroclimáticos.

In [ ]:
from agrobr.sync import nasa_power

# Clima mensal por UF (ponto central do estado)
df_clima = nasa_power.clima_uf("MT", ano=2025)
print(f"Clima MT 2025: {len(df_clima)} meses, colunas: {df_clima.columns.tolist()}")
df_clima

In [ ]:
# Clima diario por coordenada especifica (Sorriso-MT)
df_diario = nasa_power.clima_ponto(lat=-12.6, lon=-56.1, inicio="2025-01-01", fim="2025-01-31")
print(f"Diario jan/2025: {len(df_diario)} dias")
df_diario.head(10)

## 11. Cache DuckDB

O agrobr mantém cache local em DuckDB. Rode a célula abaixo 2x e compare o tempo.

In [ ]:
import time

t0 = time.time()
df, meta = datasets.preco_diario("soja", inicio="2025-01-01", return_meta=True)
elapsed = time.time() - t0

print(f"Tempo: {elapsed:.2f}s")
print(f"Cache hit: {meta.from_cache}")
print(f"Fonte: {meta.selected_source}")
print(f"Registros: {meta.records_count}")
print("\nDica: rode novamente para ver o cache hit!")

## 12. Pipeline Completo

JOIN entre preço, clima e produção com gráficos e export.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# --- Dados ---
# Preco diario
df_preco = datasets.preco_diario("soja", inicio="2025-01-01")

# Clima mensal SP (principal praca de preco)
df_clima_sp = nasa_power.clima_uf("SP", ano=2025)

# Producao por UF (CONAB com fallback inline)
try:
    df_prod = conab.safras("soja", safra="2024/25")
    prod_source = "CONAB"
except Exception:
    df_prod = pd.DataFrame(
        {
            "uf": ["MT", "PR", "RS", "GO", "MS", "BA", "MG", "SP"],
            "producao": [45200, 22100, 21800, 16200, 13400, 8500, 7200, 4300],
        }
    )
    prod_source = "CONAB (estimativa)"

print(f"Preco: {len(df_preco)} registros")
print(f"Clima SP: {len(df_clima_sp)} meses")
print(f"Producao: {len(df_prod)} UFs (fonte: {prod_source})")

In [ ]:
# --- Grafico 1: Serie temporal preco + precipitacao ---
fig, ax1 = plt.subplots(figsize=(12, 5))

ax1.plot(df_preco["data"], df_preco["valor"], color="tab:blue", linewidth=2, label="Soja (R$/sc)")
ax1.set_xlabel("Data")
ax1.set_ylabel("R$/saca 60kg", color="tab:blue")
ax1.tick_params(axis="y", labelcolor="tab:blue")

ax2 = ax1.twinx()
ax2.bar(
    df_clima_sp["mes"],
    df_clima_sp["precip_acum_mm"],
    width=20,
    alpha=0.3,
    color="tab:cyan",
    label="Precipitacao SP (mm)",
)
ax2.set_ylabel("Precipitacao (mm/mes)", color="tab:cyan")
ax2.tick_params(axis="y", labelcolor="tab:cyan")

fig.suptitle("Soja: Preco CEPEA vs Precipitacao SP (2025)", fontsize=14)
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.92))
plt.tight_layout()
plt.show()

In [ ]:
# --- Grafico 2: Producao por UF (barras horizontais) ---
top_prod = (
    df_prod[df_prod["uf"].notna()][["uf", "producao"]]
    .sort_values("producao", ascending=True)
    .tail(10)
)

fig, ax = plt.subplots(figsize=(10, 5))
ax.barh(top_prod["uf"], top_prod["producao"].astype(float), color="tab:green")
ax.set_xlabel("Producao (mil toneladas)")
ax.set_title(f"Top 10 UFs - Producao Soja ({prod_source})")
ax.grid(axis="x", alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# --- Export Parquet e CSV ---
import os
import tempfile

outdir = tempfile.mkdtemp(prefix="agrobr_")
path_parquet = os.path.join(outdir, "soja_preco.parquet")
path_csv = os.path.join(outdir, "clima_sp_2025.csv")

df_preco.to_parquet(path_parquet, index=False)
df_clima_sp.to_csv(path_csv, index=False)

print("Exportado:")
print(f"  {path_parquet}")
print(f"  {path_csv}")

## 13. Modo Async

Para pipelines de alta performance, use a API async com `asyncio.gather`.

In [ ]:
import asyncio
import time

from agrobr import cepea as cepea_async


async def fetch_multiplos():
    produtos = ["soja", "milho", "cafe"]
    t0 = time.time()
    results = await asyncio.gather(
        *[cepea_async.indicador(p, inicio="2025-01-01") for p in produtos],
        return_exceptions=True,
    )
    elapsed = time.time() - t0
    print(f"Tempo total (3 produtos em paralelo): {elapsed:.2f}s\n")
    for prod, res in zip(produtos, results):
        if isinstance(res, Exception):
            print(f"  {prod}: erro - {type(res).__name__}")
        else:
            print(f"  {prod}: {len(res)} registros")


# Compativel com Jupyter/Colab e script
try:
    loop = asyncio.get_running_loop()
    import nest_asyncio

    nest_asyncio.apply()
    loop.run_until_complete(fetch_multiplos())
except RuntimeError:
    asyncio.run(fetch_multiplos())

## 14. Links

| Recurso | Link |
|---------|------|
| PyPI | [pypi.org/project/agrobr](https://pypi.org/project/agrobr/) |
| Documentação | [www.agrobr.dev/docs](https://www.agrobr.dev/docs/) |
| GitHub | [github.com/bruno-portfolio/agrobr](https://github.com/bruno-portfolio/agrobr) |
| Changelog | [CHANGELOG.md](https://github.com/bruno-portfolio/agrobr/blob/main/CHANGELOG.md) |
| Colab | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bruno-portfolio/agrobr/blob/main/examples/agrobr_demo.ipynb) |

**agrobr v0.9.0** — 1433+ testes, ~75% cobertura, 13/13 golden tests, resiliência HTTP completa.